In [ ]:
import sys
sys.path.append("../..")

In [ ]:
from src.gershgorin.bs_gda import BS_GDA
from src.graph.graph import Graph

import numpy as np
import matplotlib.pyplot as plt
import scipy
import time
import typing

In [ ]:
def greedy_set_cover_old(coverage_subsets: typing.List[typing.Set[int]], nodes, k: int) -> typing.Tuple[list, bool]:
    """Greedy set cover algorithm.
    :param coverage_subsets: list of node subsets
    :param nodes: graph nodes
    :param k: sampling budget
    :return: sampling set, validity flag
    """
    N = len(nodes)
    # set of uncovered nodes
    uncovered = set(range(N))
    # covered sets
    covered = np.zeros(N, dtype=bool)
    # selected sets
    selected = np.zeros(N, dtype=bool)
    sampling_set = list()

    num_selected = 0
    while len(uncovered) > 0 and num_selected < k:
        max_coverage_set, i = select_max_coverage_set_old(coverage_subsets, uncovered, covered, selected)
        # remove covered nodes from uncovered nodes
        uncovered -= max_coverage_set
        # take node i into the sampling set
        sampling_set.append(i)
        selected[i] = 1
        num_selected += 1

    # init validity flag
    vf = True
    # Does each node belong to at least one coverage set?
    if len(uncovered) > 0:
        vf = False

    return sampling_set, vf

def greedy_set_cover_new(coverage_subsets: typing.List[typing.Set[int]], nodes, k: int) -> typing.Tuple[list, bool]:
    """Greedy set cover algorithm.
    :param coverage_subsets: list of node subsets
    :param nodes: graph nodes
    :param k: sampling budget
    :return: sampling set, validity flag
    """
    N = len(nodes)
    # set of uncovered nodes
    uncovered = set(range(N))
    # covered sets
    covered = np.zeros(N, dtype=bool)
    # selected sets
    selected = np.zeros(N, dtype=bool)
    sampling_set = list()

    num_selected = 0
    while len(uncovered) > 0 and num_selected < k:
        max_coverage_set, i = select_max_coverage_set_new(coverage_subsets, uncovered, covered, selected)
        # remove covered nodes from uncovered nodes
        uncovered -= max_coverage_set
        # take node i into the sampling set
        sampling_set.append(i)
        selected[i] = 1
        num_selected += 1

    # init validity flag
    vf = True
    # Does each node belong to at least one coverage set?
    if len(uncovered) > 0:
        vf = False

    return sampling_set, vf

def select_max_coverage_set_new(sets: list, uncovered: set, covered, selected) -> typing.Tuple[set, int]:
    """Selects the set that covers the most uncovered nodes.
    :param sets: list of coverage subsets
    :param uncovered: set of uncovered nodes
    :param covered: array-like binary vector that indicates which lists and their nodes are covered
    :param selected: array-like binary vector that indicates which subsets have already been selected
    :return: maximum coverage set, index of maximum set in sets
    """
    max_coverage_set = set()
    max_coverage = 0
    max_idx = None
    for node, s in enumerate(sets):
        if not (covered[node] or selected[node]):
            coverage_set = uncovered & s
            num_covered = len(coverage_set)
            if num_covered > max_coverage:
                max_coverage_set = coverage_set
                max_coverage = num_covered
                max_idx = node
            elif num_covered == 0:
                covered[node] = 1
    return max_coverage_set, max_idx

def select_max_coverage_set_old(sets: list, uncovered: set, covered, selected) -> typing.Tuple[set, int]:
    """Selects the set that covers the most uncovered nodes.
    :param sets: list of coverage subsets
    :param uncovered: set of uncovered nodes
    :param covered: array-like binary vector that indicates which lists and their nodes are covered
    :param selected: array-like binary vector that indicates which subsets have already been selected
    :return: maximum coverage set, index of maximum set in sets
    """
    max_coverage_set = set()
    max_coverage = 0
    max_idx = None
    for node, s in enumerate(sets):
        if not (covered[node] or selected[node]):
            coverage_set = s & uncovered
            num_covered = len(coverage_set)
            if num_covered > max_coverage:
                max_coverage_set = coverage_set
                max_coverage = num_covered
                max_idx = node
            elif num_covered == 0:
                covered[node] = 1
    return max_coverage_set, max_idx

In [ ]:
n = 5
ss = np.zeros(n, dtype=int)
cs = np.zeros(n, dtype=int)
uncovered = np.ones(n, dtype=bool)
coverage_subsets = [[0, 1], [1, 2, 3], [1], [2], [0]]
uncovered_set = set(np.arange(n))
sets = [set(ell) for ell in coverage_subsets]
nodes = list(range(n))
k = 1

In [ ]:
n = 10000
sets = list(range(n))
sets = [set([i]) for i in sets]
nodes = list(range(n))
k = int(0.1*n)

In [ ]:
%timeit greedy_set_cover_old(sets, nodes, k)
%timeit greedy_set_cover_new(sets, nodes, k)